此文件包含了对sd数据（来自公司员工的短信息交流数据）的coinrate 的计算

In [ ]:
%matplotlib auto
#弹出窗绘图

In [ ]:
%matplotlib inline
#控制台绘图

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import pandas as pd
import numpy as np
import random
import string
import time
import numba as nb
import networkx as nx

from scipy import stats
from mpl_toolkits.mplot3d import Axes3D


---

## 运行计算函数

导入函数

In [ ]:
%%time
#%%prun

%run ./coin_rate_calculate.ipynb
%run ./picture_by_matplotlib.ipynb

%run ./ 当前目录

%run ../ 上一级目录

---

## 分析sd数据

### 读取数据

分析文献中的sd数据（来自公司员工的短信息交流数据）

In [ ]:
pd.set_option('display.max_columns', 10,'display.max_rows', 10)  # 设置显示全部列数据行数据

In [ ]:
# 读取数据
colname=['sender','recipient','timestamp']
datel=pd.read_table('.\\data\\SD01.txt',sep='\s+',header=None,names=colname,nrows=None)

---

In [ ]:
temp1=datel[datel.sender==136].loc[:,'timestamp']
temp2=datel[datel.sender==135].loc[:,'timestamp']
pre_cr_(600,0,temp1,temp2)

In [ ]:
%%time
pre_cr_(600,0,temp1,temp2)

In [ ]:
%%time
pre_coin_r(600,0,temp1,temp2)

In [ ]:
pre_cr_center(600,0,temp1,temp2)

In [ ]:
# 获取活跃的用户的信息，发送信息大于一定值的用户
unknow=pd.value_counts(datel.loc[:,'sender'])
indx_3=unknow[unknow>500].index

---

### 计算coinrate

In [ ]:
# 生成二维空表格
neta=pd.DataFrame(np.zeros((len(indx_3),len(indx_3))),index=indx_3,columns=indx_3)
neta

In [ ]:
# 二维数据中，行对象作为列对象的pre来计算,行对象与列对象的关联占行对象的比例

# 将计算得到的coinrate填充到二维空表格
for ind in indx_3:
    for col in indx_3:
        if ind==col:
            continue
        tp1=datel[datel.sender==ind].loc[:,'timestamp']
        tp2=datel[datel.sender==col].loc[:,'timestamp']
        neta.loc[ind,col]=pre_cr_(600,0,tp1,tp2)

In [ ]:
# 更改表格字体颜色的准备
def showcolor(val):
    if val>0.5:
        color='red'
    elif val>0.2:
        color='green'
    else :
        color='blank'
    return 'color:%s'%color

In [ ]:
# 存储变量到其他文件中也可使用
new=neta.round(2)
%store new
%store indx_3

In [ ]:
# 更改表格的颜色并展示
neta.round(decimals=2).style.applymap(showcolor)

---

### 直接通过数据计算

In [ ]:
# 直接从数据中的收发信人数据计算关系
def sendemax(sd):
    showz=datel[datel.sender==sd].loc[:,'recipient']
    showzc=pd.value_counts(showz)
    #print(showzc.iloc[0]/showzc)
    rate=showzc.iloc[0]/showz.count()  #计算sender发送的信息中发给的最多的人的信息占比
    mro=showzc.index[0]
    print('sender is %5d and max rate object %5d is %.3f'%(sd,mro,rate),mro in indx_3)

In [ ]:
# 将所有联系展示
for i in indx_3:
    sendemax(i)

---

### 画出网络图

#### 设置节点和边

用箭头表示联系的流向，例如：a->b，意味着a的信息大多发给了b

In [ ]:
# 第一个数是行，第二个数是列
data_5=np.where(neta>0.5)

In [ ]:
len(indx_3)

In [ ]:
data_5

In [ ]:
da5_edge=[]
for i in range(len(data_5[0])):
    x1=data_5[0][i]
    x2=data_5[1][i]
    da5_edge.append((indx_3[x1],indx_3[x2]))
    print(x1,x2,'\tindex:\t',indx_3[x1],indx_3[x2])
print(da5_edge)

---

#### 画出有向网络图

In [ ]:
sd_g=nx.DiGraph()
sd_g.add_nodes_from(indx_3)
sd_g.add_edges_from(da5_edge)

In [ ]:
# 这个是来自SD01的数据，挑选了发送信息数大于一定值的用户,绘制出了网络图

# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# 有中文出现，需要用u'内容'

plt.figure(figsize=(20,20))
text=u'节点是发送信息数大于500的用户,\n用户数是%d,coinrate>0.5就连边'%(len(indx_3))
nx.draw(sd_g,
        with_labels=True,
        node_color='SpringGreen',
        edge_color='royalblue',
        node_size=500,
        width=1.5,
        arrowstyle='->',
#         pos=nx.circular_layout(sd),        
        pos=nx.kamada_kawai_layout(sd_g),
        )
# plt.title('network')
plt.text(0,0,text,fontdict={'color':'orangered','size':20})
plt.savefig('.\\data\\network_SD01_u500_c.5.png')
plt.show()